<a href="https://colab.research.google.com/github/aarondlc/toybox/blob/master/Iso_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install shapely
# -*- coding: utf-8 -*-
# Importamos librerias basicas
import requests
import pandas as pd
import numpy as np
from collections.abc import MutableMapping
from shapely.geometry import Polygon

# Definimos funciones propias
## Para aplanar los dict anidados
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
##Para extraer las coordenadas del poligono cortado
def extract_poly_coords(geom):
    if geom.type == 'Polygon':
        exterior_coords = geom.exterior.coords[:]
        interior_coords = []
        for interior in geom.interiors:
            interior_coords += interior.coords[:]
    elif geom.type == 'MultiPolygon':
        exterior_coords = []
        interior_coords = []
        for part in geom:
            epc = extract_poly_coords(part)  # Recursive call
            exterior_coords += epc['exterior_coords']
            interior_coords += epc['interior_coords']
    else:
        raise ValueError('Unhandled geometry type: ' + repr(geom.type))
    return {'exterior_coords': exterior_coords,
            'interior_coords': interior_coords}
 



In [0]:
'''
Script principal para descargarlos datos desde la API. Recibe un DF y devuelve el DF actualizado
sin borrar datos y un kml   
'''

## Parametros del payload
tipo = "isochrone"
#value = 600
#lat = 43.33409
#lng = -1.792069
approx = 100
mobility = "motor_vehicle"
speedType = "normal"
reduceQueue = "false"
avoidTolls = "true"
restrictedAreas = "true"
fastestRouting = "true"
concavity = 6
buffering = 3
reqId = "A57X"

rest_id = [32091,33567]
payload = [(41.3967997,2.1660627),(39.47048499999999,-0.3901581)]
tiempos = [720,900]

data = {}
Iso_tiempo = {}

for rest_id in rest_id:
    for value in tiempos:
        for lat, lng in payload:
            
            parametros = {"type" : tipo,
                          "value" : value,
                          "lat" : lat,
                          "lng" : lng,
                          "approx" : approx,
                          "mobility" : mobility,
                          "speedType" : speedType,
                          "reduceQueue" : reduceQueue,
                          "avoidTolls" : avoidTolls,
                          "restrictedAreas" : restrictedAreas,
                          "fastestRouting" : fastestRouting,
                          "concavity" : concavity,
                          "buffering" : buffering,
                          "reqId" : reqId}
            respuesta = requests.get("http://www.iso4app.net/rest/1.3/isoline.json?licKey=9F4768A8-22BF-46EE-B6EB-9A6490643CDA", parametros)
            print(respuesta)
            output = flatten(respuesta.json())
            longitud = []
            latitud = []
            for i in range(len(output['polygons'][0]['exterior'])):
                longitud.append(output['polygons'][0]['exterior'][i][0])
            for i in range(len(output['polygons'][0]['exterior'])):
                latitud.append(output['polygons'][0]['exterior'][i][1])
                Iso_tiempo[value] = list(zip(longitud, latitud))
    data[rest_id] = Iso_tiempo     
df = pd.DataFrame.from_dict(data, orient='index')     

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [0]:
#Para obtener el corte del poligono de isocronas con el de restricciones
import shapely
from shapely.geometry import Polygon
LAND = Polygon([(0, 0), (0, 20), (20, 20), (20, 0)])
diff = LAND
polygons = generate_polygons(20, 15)

for polygon in polygons:
    diff = diff.difference(polygon)
diff 

In [0]:
df.to_csv('test.csv')

In [0]:
#Para crear un kml a gusto de RM
import simplekml
kml = simplekml.Kml()
pol = kml.newpolygon(name='A Polygon')
pol.outerboundaryis = data[32091][720]
pol.innerboundaryis = []
pol.style.linestyle.color = simplekml.Color.green
pol.style.linestyle.width = 5
pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.green)
kml.save("1.kml")

In [0]:
#Para parsear los poligonos de restricciones                                                                      
#!pip install pykml
from pykml import parser

root = parser.fromstring(open('VALENCIA_ISOLINE.kml', 'rb').read())
print (root.Document.Placemark.Polygon.outerBoundaryIs.LinearRing.coordinates)


              -0.347155,39.497335,0
              -0.350545,39.49866,0
              -0.351876,39.499786,0
              -0.366325,39.506027,0
              -0.367184,39.504703,0
              -0.36802,39.503676,0
              -0.368514,39.50212,0
              -0.370016,39.500464,0
              -0.370446,39.49808,0
              -0.37508,39.499305,0
              -0.376013,39.499438,0
              -0.378148,39.499306,0
              -0.382676,39.498412,0
              -0.386324,39.497716,0
              -0.388598,39.499902,0
              -0.391174,39.502186,0
              -0.393835,39.501971,0
              -0.395122,39.502318,0
              -0.398512,39.499934,0
              -0.401173,39.499041,0
              -0.404982,39.497617,0
              -0.407546,39.496822,0
              -0.409927,39.495828,0
              -0.410056,39.494802,0
              -0.408511,39.492616,0
              -0.408855,39.492285,0
              -0.404949,39.487747,0
              -0.406623,39.48238